In [1]:
import pandas as pd

In [2]:
#takes a while to load so have in separate cell
pop_proj_full = pd.read_csv("/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/projected_population_densities/colombia/anexo-proyecciones-poblacion-Municipal_2027-2035.csv", skiprows =6 )
print(pop_proj_full.columns)

/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_34259/3791409054.py:2: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308) have mixed types. Sp

Index(['DP', 'DPNOM', 'DPMP', 'MPIO', 'AÑO', 'ÁREA GEOGRÁFICA', 'Hombres_0',
       'Hombres_1', 'Hombres_2', 'Hombres_3',
       ...
       'Total_94', 'Total_95', 'Total_96', 'Total_97', 'Total_98', 'Total_99',
       'Total_100 y más', 'Total Hombres', 'Total Mujeres', 'Total'],
      dtype='object', length=312)


In [3]:
#sorting the projected populations
pop_proj = pop_proj_full[['DPMP', 'MPIO', 'AÑO', 'ÁREA GEOGRÁFICA', 'Total']].copy()
pop_proj = pop_proj[pop_proj["ÁREA GEOGRÁFICA"]=='Total']
print(pop_proj["AÑO"].unique())
pop_proj = pop_proj[pop_proj["AÑO"]==2035]
pop_proj = pop_proj.rename(columns={'DPMP':'MUN_CODE', 'MPIO':'MUN', 'Total':'POP'})
pop_proj = pop_proj.drop(columns=['AÑO',"ÁREA GEOGRÁFICA" ]).reset_index()
pop_proj["POP"] = pop_proj["POP"].str.replace(',', "")
pop_proj["MUN_CODE"] = pop_proj["MUN_CODE"].astype(str)
pop_proj["MUN_CODE"] = pop_proj["MUN_CODE"].str.strip()
pop_proj["MUN"] = pop_proj["MUN"].str.upper()
# print(pop_proj)

#calc the projected densities using current data set of area
pop_now = pd.read_csv("/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/vulnerability_indices/colombian_vulnerability_by_municipality_corrected.csv").iloc[:,:3]
pop_proj['POP_DENS'] = pop_proj["POP"].astype(float)/pop_now["MUN_AREA"]  
pop_proj = pop_proj[['MUN_CODE', "MUN", "POP_DENS"]]


[2027 2028 2029 2030 2031 2032 2033 2034 2035]


In [4]:
#append current SVI to the projected population 
svi = pd.read_csv("/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/vulnerability_indices/colombian_vulnerability_by_municipality_corrected.csv")
svi["MUN_CODE"] = svi["MUN_CODE"].astype(str)

pop_proj_svi = pop_proj.merge(svi,on = 'MUN_CODE', how = 'outer')
# print(pop_proj_svi)
    
#merge with suitability data
for i in ['245', '585']:
    suitability_mean_median = pd.read_csv("/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/"+i+"/model1/suitability_mean_median.csv").iloc[:,1:]
    suitability_mean_median["MUN_CODE"] = suitability_mean_median["MUN_CODE"].astype(str)
    pop_proj_svi_suit = pop_proj_svi.merge(suitability_mean_median,on = 'MUN_CODE', how = 'outer')
    pop_proj_svi_suit = pop_proj_svi_suit.drop(columns = ["MUN_x", "MUN_AREA"])
    pop_proj_svi_suit = pop_proj_svi_suit[['MUN_CODE', 'MUN_y', 'SVI', 'POP_DENS',"SUITABILITY_MEAN", "SUITABILITY_MEDIAN"]]
    pop_proj_svi_suit = pop_proj_svi_suit.rename(columns = {"MUN_y": "MUN"})
    print(pop_proj_svi_suit.columns)
    # pop_proj_svi_suit.to_csv("/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/"+i+"/model1/rf_projection_input.csv")

Index(['MUN_CODE', 'MUN', 'SVI', 'POP_DENS', 'SUITABILITY_MEAN',
       'SUITABILITY_MEDIAN'],
      dtype='object')
Index(['MUN_CODE', 'MUN', 'SVI', 'POP_DENS', 'SUITABILITY_MEAN',
       'SUITABILITY_MEDIAN'],
      dtype='object')
